In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Here, we develop tools to pick up data needed for calibration

In [2]:
from LeCheapEyeTracker import Client, Server, Stimulation

import numpy as np
import matplotlib.pyplot as plt
import time

import cv2
from vispy import gloo, app

## First, we need to pick up stimuli x_positions.

In [3]:
class NeoStimulation(Stimulation):
    def get_stimulus(self, t0, t):
        i = int((t-t0)//self.transition_lag)
        stim_x, stim_y = self.tabPos[i]
        return self.draw_stimulus((stim_x, stim_y)), stim_x

In [4]:
class NeoClient(Client):
    def __init__(self, et, timeline, downscale, stim_type = 'calibration'):
        self.downscale = downscale
        Client.__init__(self, et, timeline, self.downscale, stim_type)
        self.stimulation = NeoStimulation(self.width//self.downscale, self.height//self.downscale, stim_type = stim_type)
        self.stims_posX = np.array([])
        self.stims_time = np.array([])
        
    def on_timer(self, event):
        if time.time()-self.start < self.timeline.max():
            image, x = self.stimulation.get_stimulus(t0 = self.start, t = time.time())
            self.stims_posX = np.append(self.stims_posX, x)
            self.stims_time = np.append(self.stims_time, time.time()-self.start)
            self.program['texture'][...] = image.astype(np.uint8).reshape((self.height//self.downscale, self.width//self.downscale, 3))
        else:
            self.close()
            
        try:
            frame = self.et.cam.grab()
            if not frame is None:
                res, t0 = self.et.process_frame(frame.copy(), self.et.clock())
                self.et.eye_pos.append([res, t0])
        except Exception as e:
            if not self.et is None: print('could not grab a frame / detect the eye position', e)
        self.update()

In [5]:
fps = 30
screen = NeoClient(et=Server(), timeline=np.linspace(0, 7., 3.*fps), downscale = 2)
screen.app.run()

window size :  (2560, 1440)
[[ 2814221.75      3340618.        3860712.       ..., -3746001.       -3599436.5
  -3451485.      ]
 [ 2715890.        3234330.        3751732.75     ..., -3757115.25
  -3605549.       -3453532.      ]
 [ 2611275.25      3116036.25      3625952.25     ..., -3754691.25
  -3598974.       -3442701.      ]
 ..., 
 [  820482.25       841611.4375     860223.0625   ...,  -637504.3125
   -843282.0625   -1018120.      ]
 [  881505.6875     906611.1875     929112.5      ...,  -359577.875
   -537335.125     -686421.6875  ]
 [  952143.875      979847.6875    1004554.3125   ...,  -100618.9375
   -248159.734375  -368359.15625 ]]
[[ 2587011.          3065989.          3551570.5        ..., -3412450.
  -3247090.         -3091669.25      ]
 [ 2491035.75        2947310.5         3417389.75       ..., -3401935.25
  -3232275.75       -3072119.5       ]
 [ 2390574.75        2823194.75        3276661.5        ..., -3372776.5
  -3197019.25       -3031345.        ]
 ..., 
 [ 12836

/usr/local/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


In [ ]:
print (screen.stims_posX)

In [ ]:
print (screen.stims_time)

## Time to pick up eye x_positions

In [ ]:
print (screen.et.eye_pos)

Good !

In [1]:
! git commit -am 'We have targets and eye positions, now'

[master febaafb] We have targets and eye positions, now
 3 files changed, 878 insertions(+), 86 deletions(-)


In [2]:
! git push

Counting objects: 8, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (8/8), 17.18 KiB | 0 bytes/s, done.
Total 8 (delta 5), reused 0 (delta 0)
To https://github.com/meduz/LeCheapEyeTracker
   2e89088..febaafb  master -> master
